# Translate columns from Hebrew to English

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

import warnings
warnings.filterwarnings("ignore")

In [122]:
population_o = pd.read_excel('2018.xlsx',sheet_name='population')
budget_o = pd.read_excel('2018.xlsx',sheet_name='budget')

In [123]:
population = population_o.copy()
budget = budget_o.copy()

### Merge Tables by Authority Code


In [124]:
# Rename columns to a standardized English name
budget = budget.rename(columns={'סמל הרשות': 'authority_code'})
population = population.rename(columns={'כללי - סמל הרשות': 'authority_code'})
# Merge tables by authority_code, keeping all columns
merged_df = pd.merge(budget, population, on='authority_code', how='outer')

# Optional: Display basic information about the merged dataset
print("\nMerged dataset info:")
merged_df.info()


Merged dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Columns: 397 entries, שם  הרשות_x to בחירות - בחירות לכנסת ה-22
17/9/19 - אחוזי הצבעה
dtypes: float64(54), int64(74), object(269)
memory usage: 828.2+ KB


#### Column names

#### column_mapping

 עכשיו אנחנו מכסים את כל 397 העמודות. הנה סיכום הקטגוריות הראשיות שיצרנו:

- auth_ (מידע בסיסי על הרשות)
- budget_reg_inc_ (תקציב רגיל - הכנסות)
- budget_reg_exp_ (תקציב רגיל - הוצאות)
- budget_dev_ (תקציב פיתוח)
- arnona_ (ארנונה)
- demo_ (דמוגרפיה)
- edu_ (חינוך)
- health_ (בריאות)
- employ_ (תעסוקה ורווחה)
- construct_ (בנייה ודיור)
- infra_ (תשתיות)
- transport_ (תחבורה)
- crime_ (פשיעה ומשפט)
- water_ (מים)
- waste_ (פסולת)
- land_ (שימושי קרקע)
- elect_ (בחירות)
- index_ (מדדים ודירוגים)
- region_ (מועצות אזוריות)
- migrate_ (הגירה)
- family_ (משפחה ונישואין)
- inst_ (אוכלוסייה במוסדות)
- welfare_ (רווחה)

In [125]:
def create_complete_column_mapping():
    """
    Complete hierarchical column mapping for all 397 columns
    Structure: category_subcategory_metric_type
    """
    mapping = { 

        ### BASIC AUTHORITY INFO (auth_) ###
        'שם הרשות_x': 'auth_name',
        'authority_code': 'auth_code',
        'מעמד מוניציפאלי': 'auth_status',

        ### BUDGET INCOME - REGULAR (budget_reg_inc_) ###
        'סה"כ תקבולים (תקציב רגיל ותקציב בלתי רגיל)': 'budget_total_income',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב- סה"כ הכנסות בתקציב הרגיל': 'budget_reg_total_income',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-הכנסות לנפש בתקציב הרגיל': 'budget_reg_income_per_capita',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-מיסים ומענקים': 'budget_reg_inc_tax_grants',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-שירותים מקומיים': 'budget_reg_inc_local_services',

        ### BUDGET INCOME - GOVERNMENT SERVICES (budget_reg_inc_gov_) ###
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-שירותים ממלכתיים- סה"כ שירותים ממלכתיים': 'budget_reg_inc_gov_services_total',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-שירותים ממלכתיים- חינוך': 'budget_reg_inc_gov_education',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-שירותים ממלכתיים- רווחה': 'budget_reg_inc_gov_welfare',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-שירותים ממלכתיים - תרבות': 'budget_reg_inc_gov_culture',

        ### BUDGET INCOME - ENTERPRISES (budget_reg_inc_enterprise_) ###
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-מפעלים- סה"כ מפעלים': 'budget_reg_inc_enterprises_total',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב- מפעלים- מפעל המים': 'budget_reg_inc_water_enterprise',
        'תקבולים בתקציב הרגיל- הכנסות לפי פרקי תקציב-תקבולים בלתי רגילים': 'budget_reg_inc_irregular',

        ### BUDGET INCOME - BY ECONOMIC CATEGORY (budget_reg_inc_econ_) ###
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי- סה"כ הכנסות בתקציב הרגיל': 'budget_reg_inc_econ_total',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-סה"כ הכנסות עצמיות': 'budget_reg_inc_econ_independent_total',

        ### BUDGET INCOME - ARNONA (budget_reg_inc_arnona_) ###
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- סה"כ הכנסות מארנונה': 'budget_reg_inc_arnona_total',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה - \'למגורים: סה"כ גביות': 'budget_reg_inc_arnona_residential_collection',

        ### BUDGET INCOME - ARNONA RESIDENTIAL (budget_reg_inc_arnona_res_) ###
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- \'למגורים: סה"כ חיובים נטו': 'budget_reg_inc_arnona_res_charges_net',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- \'למגורים: יחס גבייה ב-% לכלל החיובים': 'budget_reg_inc_arnona_res_collection_rate_pct',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה - \'למגורים: גבייה השנה (קרן + ריבית)': 'budget_reg_inc_arnona_res_collection_current',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה - \'למגורים: חיוב בשנת החשבון': 'budget_reg_inc_arnona_res_charges_current',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- \'למגורים: יחס גבייה ב-% לחיוב השנה': 'budget_reg_inc_arnona_res_collection_rate_current_pct',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- \'למגורים: סה"כ פטורים, שחרורים והנחות': 'budget_reg_inc_arnona_res_exemptions_total',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- \'למגורים: גביית פיגורים (קרן + ריבית)': 'budget_reg_inc_arnona_res_collection_arrears',

        ### BUDGET INCOME - ARNONA NON-RESIDENTIAL (budget_reg_inc_arnona_nonres_) ###
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- \'לא למגורים (גבייה)': 'budget_reg_inc_arnona_nonres_collection',

        ### BUDGET INCOME - SELF GENERATED (budget_reg_inc_self_) ###
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות - הכנסות מחינוך': 'budget_reg_inc_self_education',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות- הכנסות מרווחה': 'budget_reg_inc_self_welfare',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות - יתר הכנסות עצמיות': 'budget_reg_inc_self_other',

        ### BUDGET INCOME - GOVERNMENT TRANSFERS (budget_reg_inc_gov_transfer_) ###
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה- סה"כ הכנסות מהממשלה': 'budget_reg_inc_gov_transfer_total',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה-ממשרד החינוך': 'budget_reg_inc_gov_transfer_education',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה-ממשרד הרווחה': 'budget_reg_inc_gov_transfer_welfare',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה-מענק כללי': 'budget_reg_inc_gov_transfer_general',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה-מענקים מיועדים': 'budget_reg_inc_gov_transfer_designated',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה-מענקים מיוחדים': 'budget_reg_inc_gov_transfer_special',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-\'הכנסות מהממשלה-מענקים להקטנת גירעון מצטבר': 'budget_reg_inc_gov_transfer_deficit',
        'תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-מלוות לאיזון': 'budget_reg_inc_gov_transfer_balance_loans',

        ### DEVELOPMENT BUDGET INCOME (budget_dev_inc_) ###
        'תקבולים בתקציב הבלתי רגיל-סה"כ הכנסות בתקציב בלתי רגיל': 'budget_dev_inc_total',
        'תקבולים בתקציב הבלתי רגיל- השתתפות בעלי נכסים': 'budget_dev_inc_property_owners',
        'תקבולים בתקציב הבלתי רגיל- השתתפות משרדי ממשלה': 'budget_dev_inc_gov',
        'תקבולים בתקציב הבלתי רגיל-מלוות': 'budget_dev_inc_loans',
        'תקבולים בתקציב הבלתי רגיל-הכנסות אחרות': 'budget_dev_inc_other',
        ### TOTAL EXPENSES (budget_total_exp_) ###
        'סה"כ תשלומים (תקציב רגיל ותקציב בלתי רגיל)': 'budget_total_exp',
        'תשלומים בתקציב הרגיל -סה"כ הוצאות בתקציב הרגיל': 'budget_reg_exp_total',
        'תשלומים בתקציב הרגיל - הוצאה לנפש בתקציב הרגיל': 'budget_reg_exp_per_capita',

        ### REGULAR BUDGET EXPENSES BY CATEGORY (budget_reg_exp_cat_) ###
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב-הנהלה וכלליות': 'budget_reg_exp_cat_management',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -שירותים מקומיים': 'budget_reg_exp_cat_local_services',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -שירותים ממלכתיים- סה"כ שירותים ממלכתיים': 'budget_reg_exp_cat_gov_services_total',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -שירותים ממלכתיים-חינוך': 'budget_reg_exp_cat_education',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -שירותים ממלכתיים- רווחה': 'budget_reg_exp_cat_welfare',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -שירותים ממלכתיים- תרבות': 'budget_reg_exp_cat_culture',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -מפעלים -סה"כ מפעלים': 'budget_reg_exp_cat_enterprises_total',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -מפעלים-מפעל המים': 'budget_reg_exp_cat_water',
        'תשלומים בתקציב הרגיל -הוצאות לפי פרקי תקציב -תשלומים בלתי רגילים': 'budget_reg_exp_cat_irregular',
        'תשלומים בתקציב הרגיל - \'סה"כ הוצאות בתקציב הרגיל': 'budget_reg_exp_cat_total',

        ### REGULAR BUDGET EXPENSES BY ECONOMIC CLASSIFICATION (budget_reg_exp_econ_) ###
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות לעלות עבודה -סה"כ הוצאות לעלות עבודה': 'budget_reg_exp_econ_salary_total',
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות לעלות עבודה -לחינוך': 'budget_reg_exp_econ_salary_education',
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות לעלות עבודה -לרווחה': 'budget_reg_exp_econ_salary_welfare',

        ### REGULAR BUDGET EXPENSES BY ECONOMIC CLASSIFICATION (budget_reg_exp_econ_) ###
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות לפעולות': 'budget_reg_exp_econ_operations',
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות לפירעון מלוות': 'budget_reg_exp_econ_loan_repayment',
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות מימון': 'budget_reg_exp_econ_financing',
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-\'העברות והוצאות חד-פעמיות (כולל כיסוי גירעון מצטבר)': 'budget_reg_exp_econ_transfers_onetime',
        'תשלומים בתקציב הרגיל -הוצאות לפי סיווג כלכלי-הוצאות כלליות, השתתפויות ותמיכות': 'budget_reg_exp_econ_general_support',

        ### DEVELOPMENT BUDGET EXPENSES (budget_dev_exp_) ###
        'תשלומים בתקציב הבלתי רגיל-סה"כ הוצאות בתקציב בלתי רגיל': 'budget_dev_exp_total',
        'תשלומים בתקציב הבלתי רגיל- \'הוצאות לעבודות שבוצעו במשך השנה': 'budget_dev_exp_projects',
        'תשלומים בתקציב הבלתי רגיל-\'הוצאות להעברת מלוות': 'budget_dev_exp_loan_transfers',
        'תשלומים בתקציב הבלתי רגיל-\'הוצאות אחרות': 'budget_dev_exp_other',

        ### BUDGET BALANCE (budget_balance_) ###
        'עודף/גירעון-\'עודף/גירעון בתקציב הרגיל השנה': 'budget_balance_regular_current',
        'עודף/גירעון-\'עודף/גירעון בתקציב הרגיל השנה ללא קבלת מלוות לאיזון': 'budget_balance_regular_no_loans',
        'עודף/גירעון-עודף/גירעון מצטבר בתקציב הרגיל לסוף שנה': 'budget_balance_regular_accumulated',
        'עומס מלוות לפירעון בסוף השנה (כולל הפרשי הצמדה)': 'budget_loan_burden_total',

        ### ARNONA AREA BY PROPERTY TYPE (arnona_area_) ###
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)-סה"כ חיוב ארנונה (שטח באלפי מ"ר)': 'arnona_area_total_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)-למגורים': 'arnona_area_residential_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)-למשרדים, שירותים ומסחר': 'arnona_area_commercial_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)-לתעשייה': 'arnona_area_industry_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- למלאכה': 'arnona_area_craft_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לבנקים וחברות ביטוח': 'arnona_area_banks_insurance_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לבתי מלון': 'arnona_area_hotels_sqm',

        ### ARNONA AREA BY PROPERTY TYPE (arnona_area_) -   ###
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לחניונים': 'arnona_area_parking_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לקרקע תפוסה': 'arnona_area_occupied_land_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לקרקע תפוסה המשמשת לעריכת אירועים': 'arnona_area_events_land_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לקרקע תפוסה במפעל עתיר שטח': 'arnona_area_large_industry_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לאדמה חקלאית': 'arnona_area_agriculture_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- למבנה חקלאי': 'arnona_area_agriculture_building_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- למגורים שאינם בשימוש': 'arnona_area_residential_unused_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לאדמת בניין': 'arnona_area_construction_land_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- למערכת סולרית הממוקמת על גג נכס': 'arnona_area_solar_roof_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- למערכת סולרית שאינה ממוקמת על גג נכס': 'arnona_area_solar_ground_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- לקרקע תפוסה למערכת סולרית': 'arnona_area_solar_occupied_land_sqm',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (שטח באלפי מ"ר)- ליתר סוגי הנכסים': 'arnona_area_other_sqm',

        ### ARNONA CHARGES BY PROPERTY TYPE (arnona_charge_) ###
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- סה"כ חיוב ארנונה (באלפי ₪)': 'arnona_charge_total_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למגורים': 'arnona_charge_residential_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למשרדים, שירותים ומסחר': 'arnona_charge_commercial_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לתעשייה': 'arnona_charge_industry_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למלאכה': 'arnona_charge_craft_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לבנקים וחברות ביטוח': 'arnona_charge_banks_insurance_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לבתי מלון': 'arnona_charge_hotels_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לחניונים': 'arnona_charge_parking_nis',
            
        ### ARNONA CHARGES BY PROPERTY TYPE (arnona_charge_) ###
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לקרקע תפוסה': 'arnona_charge_occupied_land_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לקרקע תפוסה המשמשת לעריכת אירועים': 'arnona_charge_events_land_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לקרקע תפוסה במפעל עתיר שטח': 'arnona_charge_large_industry_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לאדמה חקלאית': 'arnona_charge_agriculture_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למבנה חקלאי': 'arnona_charge_agriculture_building_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למגורים שאינם בשימוש': 'arnona_charge_residential_unused_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לאדמת בניין': 'arnona_charge_construction_land_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למערכת סולרית הממוקמת על גג נכס': 'arnona_charge_solar_roof_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- למערכת סולרית שאינה ממוקמת על גג נכס': 'arnona_charge_solar_ground_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- לקרקע תפוסה למערכת סולרית': 'arnona_charge_solar_occupied_land_nis',
        'חיובי ארנונה לפי סוג נכס-חיוב ארנונה (באלפי ₪)- ליתר סוגי הנכסים': 'arnona_charge_other_nis',

        ### GENERAL INFO (auth_general_) ###
        'שם הרשות_y': 'auth_general_name_alt',
        'כללי - מחוז': 'auth_general_district', 
        'כללי - מעמד מוניציפלי': 'auth_general_municipal_status',
        'כללי - 2015 - מרחק מגבול מחוז תל אביב (ק"מ)': 'auth_general_distance_tlv_km',
        'כללי - שנת קבלת מעמד מוניציפלי': 'auth_general_establishment_year',
        'כללי - מרחב ימי, רשויות מקומיות הגובלות בחוף הים': 'auth_general_coastal_status',
        'כללי - מספר חברי מועצה': 'auth_general_council_members',
        'כללי - סמל ועדת תכנון ובנייה': 'auth_general_planning_committee_code',
        'כללי - שם ועדת תכנון ובנייה': 'auth_general_planning_committee_name',

        ### DEMOGRAPHICS - GENERAL (demo_general_) ###
        'כללי - שטח (קמ"ר)': 'demo_general_area_km2',
        'דמוגרפיה - צפיפות אוכלוסייה לקמ\'\'ר ביישובים שמנו 5,000 תושבים ויותר': 'demo_general_density_per_km2',
        'דמוגרפיה - סה"כ אוכלוסייה בסוף השנה': 'demo_general_population_total',
        'דמוגרפיה - יהודים ואחרים (אחוזים)': 'demo_general_jews_others_pct',
        'דמוגרפיה - יהודים (אחוזים מתוך יהודים ואחרים)': 'demo_general_jews_pct',
        'דמוגרפיה - ערבים (אחוזים)': 'demo_general_arab_pct',
        'דמוגרפיה - מוסלמים (אחוזים מתוך האוכלוסייה הערבית)': 'demo_general_muslim_pct',
        'דמוגרפיה - נוצרים (אחוזים מתוך האוכלוסייה הערבית)': 'demo_general_christian_pct',
        'דמוגרפיה - דרוזים (אחוזים מתוך האוכלוסייה הערבית)': 'demo_general_druze_pct',
        'דמוגרפיה - סה"כ גברים בסוף השנה (אלפים)': 'demo_general_male_thousands',
        'דמוגרפיה - סה"כ נשים בסוף השנה (אלפים)': 'demo_general_female_thousands',
        ### DEMOGRAPHICS - AGE GROUPS (demo_age_) ###
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 4-0': 'demo_age_0_4_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 9-5': 'demo_age_5_9_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 14-10': 'demo_age_10_14_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 19-15': 'demo_age_15_19_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 29-20': 'demo_age_20_29_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 44-30': 'demo_age_30_44_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 59-45': 'demo_age_45_59_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 64-60': 'demo_age_60_64_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 65 ומעלה': 'demo_age_65plus_pct',

        ### DEMOGRAPHICS - AGE GROUPS SPECIAL (demo_age_special_) ###
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 17-0': 'demo_age_special_0_17_pct',
        'דמוגרפיה - אחוז באוכלוסייה בסוף השנה - בני 75 ומעלה': 'demo_age_special_75plus_pct',
        'דמוגרפיה - יחס תלות (לאלף תושבים בלתי תלויים)': 'demo_age_special_dependency_ratio',

        ### DEMOGRAPHICS - VITAL STATISTICS (demo_vital_) ###
        'דמוגרפיה - לידות חי': 'demo_vital_births',
        'דמוגרפיה - פטירות תינוקות (ממוצע)': 'demo_vital_infant_mortality',
        'דמוגרפיה - שיעור פטירות תינוקות לאלף לידות חי': 'demo_vital_infant_mortality_rate',
        'דמוגרפיה - פטירות': 'demo_vital_deaths',
        'דמוגרפיה - ריבוי טבעי סה"כ': 'demo_vital_natural_increase',
        'דמוגרפיה - ריבוי טבעי לאלף תושבים': 'demo_vital_natural_increase_rate',
        'דמוגרפיה - אחוז גידול האוכלוסייה לעומת שנה קודמת': 'demo_vital_population_growth_pct',

        ### DEMOGRAPHICS - IMMIGRATION (demo_immig_) ###
        'דמוגרפיה - אחוז עולי 1990+ מסך האוכלוסייה': 'demo_immig_post1990_pct',
        'דמוגרפיה - השתקעות ראשונה של עולים': 'demo_immig_new_immigrants',
        'דמוגרפיה - אחוז עולים בהשתקעות ראשונה - בני 17-0': 'demo_immig_age_0_17_pct',
        'דמוגרפיה - אחוז עולים בהשתקעות ראשונה - בני 44-18': 'demo_immig_age_18_44_pct',
        'דמוגרפיה - אחוז עולים בהשתקעות ראשונה - בני 64-45': 'demo_immig_age_45_64_pct',
        'דמוגרפיה - אחוז עולים בהשתקעות ראשונה - בני 65 ומעלה': 'demo_immig_age_65plus_pct',

        ### DEMOGRAPHICS - INTERNAL MIGRATION (demo_migrate_) ###
        'דמוגרפיה - מאזן הגירה ביישוב סה"כ': 'demo_migrate_balance_total',
        'דמוגרפיה - מאזן הגירה פנימית': 'demo_migrate_balance_internal',
        'דמוגרפיה - נכנסים מיישובים אחרים - סה"כ': 'demo_migrate_incoming_total',
        'דמוגרפיה - נכנסים מיישובים אחרים - גברים': 'demo_migrate_incoming_male',

        ### DEMOGRAPHICS - INTERNAL MIGRATION (demo_migrate_) ###
        'דמוגרפיה - נכנסים מיישובים אחרים - נשים': 'demo_migrate_incoming_female',
        'דמוגרפיה - נכנסים מיישובים אחרים - עד גיל 14': 'demo_migrate_incoming_age_0_14',
        'דמוגרפיה - נכנסים מיישובים אחרים - גיל 29-15': 'demo_migrate_incoming_age_15_29',
        'דמוגרפיה - נכנסים מיישובים אחרים - גיל 64-30': 'demo_migrate_incoming_age_30_64',
        'דמוגרפיה - נכנסים מיישובים אחרים - גיל 65 ומעלה': 'demo_migrate_incoming_age_65plus',
        'דמוגרפיה - נכנסים מיישובים אחרים - סה"כ.1': 'demo_migrate_outgoing_total',
        'דמוגרפיה - נכנסים מיישובים אחרים - גברים.1': 'demo_migrate_outgoing_male',
        'דמוגרפיה - נכנסים מיישובים אחרים - נשים.1': 'demo_migrate_outgoing_female',
        'דמוגרפיה - נכנסים מיישובים אחרים - עד גיל 14.1': 'demo_migrate_outgoing_age_0_14',
        'דמוגרפיה - נכנסים מיישובים אחרים - גיל 29-15.1': 'demo_migrate_outgoing_age_15_29',
        'דמוגרפיה - נכנסים מיישובים אחרים - גיל 64-30.1': 'demo_migrate_outgoing_age_30_64',
        'דמוגרפיה - נכנסים מיישובים אחרים - גיל 65 ומעלה.1': 'demo_migrate_outgoing_age_65plus',

        ### DEMOGRAPHICS - MARRIAGE AND DIVORCE (demo_marital_) ###
        'דמוגרפיה - 2017 - נישאים - סה"כ': 'demo_marital_marriages_total',
        'דמוגרפיה - 2017 - נישאים - שיעור ל-1,000 תושבים לא נשואים בני 15 ומעלה': 'demo_marital_marriage_rate',
        'דמוגרפיה - 2017 - מתגרשים - סה"כ': 'demo_marital_divorces_total',
        'דמוגרפיה - 2017 - מתגרשים - שיעור ל-1,000 תושבים נשואים בני 15 ומעלה': 'demo_marital_divorce_rate',

        ### DEMOGRAPHICS - INSTITUTIONS (demo_inst_) ###
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - סך מוסדות לינה ברשות': 'demo_inst_facilities_total',
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - סך דיירי מוסדות לינה ברשות': 'demo_inst_residents_total',
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - אחוז דיירים בבתי אבות ודיור מוגן': 'demo_inst_elderly_home_pct',

        ### DEMOGRAPHICS - INSTITUTIONS (demo_inst_) ##
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - אחוז דיירים במוסדות חינוך': 'demo_inst_education_pct',
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - אחוז דיירים במוסדות דת': 'demo_inst_religious_pct',
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - אחוז דיירים במוסדות אשפוז כולל הוסטלים': 'demo_inst_medical_pct',
        'דמוגרפיה - אוכלוסייה במוסדות לינה (אחוז מתוך אוכלוסיית הרשות) - אחוז דיירים במוסדות אחרים': 'demo_inst_other_pct',

        ### HEALTH - GENERAL (health_general_) ###
        'בריאות - שיעור פטירות מתוקנן ל-1,000 תושבים': 'health_general_mortality_rate',
        'בריאות - תוחלת חיים בלידה סך הכל - ברשויות מעל 50,000 תושבים': 'health_general_life_expectancy',
        'בריאות - שיעור פריון כולל': 'health_general_fertility_rate',
        'בריאות - מספר מקרי סוכרת ממוצע לשנה': 'health_general_diabetes_cases',
        'בריאות - שיעור מקרי סוכרת מתוקנן ל-1,000 תושבים': 'health_general_diabetes_rate',

        ### HEALTH - OBESITY (health_obesity_) ###
        'בריאות - עודף משקל והשמנה כיתה א - אחוזים': 'health_obesity_grade1_pct',
        'בריאות - עודף משקל והשמנה כיתה ז - אחוזים': 'health_obesity_grade7_pct',

        ### HEALTH - CANCER (health_cancer_) ###
        'בריאות - 2015-2011 - מספר מקרי סרטן ממוצע לשנה, גברים': 'health_cancer_male_cases',
        'בריאות - 2015-2011 - שיעור סרטן מכל הסוגים מתוקנן ל-100,000 תושבים, גברים': 'health_cancer_male_rate',
        'בריאות - 2015-2011 - מספר מקרי סרטן ממוצע לשנה, נשים': 'health_cancer_female_cases',
        'בריאות - 2015-2011 - שיעור סרטן מכל הסוגים מתוקנן ל-100,000 תושבים, נשים': 'health_cancer_female_rate',

        ### WELFARE AND EMPLOYMENT - UNEMPLOYMENT (welfare_unemp_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי דמי אבטלה (ממוצע חודשי) - סה"כ': 'welfare_unemp_benefits_total',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי דמי אבטלה (ממוצע חודשי) - גברים': 'welfare_unemp_benefits_male',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי הכשרה מקצועית סה"כ': 'welfare_unemp_training_total',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - גיל ממוצע של מקבלי דמי אבטלה (לא כולל חיילים)': 'welfare_unemp_avg_age',

        ### WELFARE AND EMPLOYMENT - UNEMPLOYMENT BENEFITS (welfare_unemp_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - דמי אבטלה ממוצעים ליום (ש"ח)': 'welfare_unemp_daily_payment_nis',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי קצבאות זקנה ושאירים סה"כ (סוף שנה)': 'welfare_elderly_benefits_total',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז מקבלי השלמת הכנסה מבין מקבלי קצבאות זקנה ושאירים': 'welfare_elderly_income_supplement_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי גמלת הבטחת הכנסה (נפשות, במשך השנה)': 'welfare_income_support_count',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי גמלאות סיעוד (סוף שנה)': 'welfare_nursing_benefits_count',

        ### WELFARE - DISABILITY BENEFITS (welfare_disability_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי גמלאות נכות כללית (סוף שנה)': 'welfare_disability_general_count',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי גמלאות ניידות (סוף שנה)': 'welfare_disability_mobility_count',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מקבלי גמלאות נכות מעבודה ותלויים (סוף שנה)': 'welfare_disability_work_count',

        ### HEALTH - INSURANCE (health_insurance_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז המבוטחים בקופ"ח מתוך סך כל המבוטחים - כללית': 'health_insurance_clalit_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז המבוטחים בקופ"ח מתוך סך כל המבוטחים - מכבי': 'health_insurance_maccabi_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז המבוטחים בקופ"ח מתוך סך כל המבוטחים - מאוחדת': 'health_insurance_meuhedet_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז המבוטחים בקופ"ח מתוך סך כל המבוטחים - לאומית': 'health_insurance_leumit_pct',

        ### WELFARE - CHILD BENEFITS (welfare_child_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר משפחות של מקבלי קצבאות בגין ילדים (סוף שנה) - סה"כ': 'welfare_child_families_total',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר משפחות של מקבלי קצבאות בגין ילדים (סוף שנה) - עם 2-1 ילדים במשפחה': 'welfare_child_families_1_2_kids',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר משפחות של מקבלי קצבאות בגין ילדים (סוף שנה) - עם 4-3 ילדים במשפחה': 'welfare_child_families_3_4_kids',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר משפחות של מקבלי קצבאות בגין ילדים (סוף שנה) - עם 5 ילדים ויותר במשפחה': 'welfare_child_families_5plus_kids',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר ילדים שבגינם שולמו קצבאות - במשפחות עם 2-1 ילדים': 'welfare_child_count_1_2_kids',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר ילדים שבגינם שולמו קצבאות - במשפחות עם 4-3 ילדים': 'welfare_child_count_3_4_kids',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - מספר ילדים שבגינם שולמו קצבאות - במשפחות עם 5 ילדים ויותר': 'welfare_child_count_5plus_kids',
            
        ### EMPLOYMENT - SALARY (employ_salary_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - שכר ממוצע לחודש של שכירים (ש"ח) - כולל': 'employ_salary_avg_total_nis',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - שכר ממוצע לחודש של שכירים (ש"ח) - גברים': 'employ_salary_avg_male_nis',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - שכר ממוצע לחודש של שכירים (ש"ח) - נשים': 'employ_salary_avg_female_nis',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - אחוז שינוי ריאלי לעומת שנה קודמת של שכר ממוצע לחודש של שכירים - כולל': 'employ_salary_change_total_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז שינוי ריאלי לעומת שנה קודמת של שכר ממוצע לחודש של שכירים - גברים': 'employ_salary_change_male_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - אחוז שינוי ריאלי לעומת שנה קודמת של שכר ממוצע לחודש של שכירים - נשים': 'employ_salary_change_female_pct',

        ### EMPLOYMENT - WORKERS (employ_workers_) ###
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - מספר השכירים': 'employ_workers_salaried_count',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - אחוז השכירים המשתכרים עד שכר מינימום': 'employ_workers_min_wage_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - מספר העצמאים': 'employ_workers_self_employed_count',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - הכנסה ממוצעת לחודש של העצמאים (ש"ח)': 'employ_workers_self_employed_income_nis',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - אחוז שינוי ראלי של הכנסה ממוצעת לחודש של העצמאים לעומת שנה קודמת': 'employ_workers_self_employed_income_change_pct',
        'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - אחוז העצמאים המשתכרים עד מחצית השכר הממוצע': 'employ_workers_self_employed_low_income_pct',
            
        ### EMPLOYMENT - INEQUALITY (employ_inequality_) ###
       'שכר ורווחה - מקבלי דמי אבטלה (ממוצע חודשי) - 2017 - מדד אי-השוויון שכירים (מדד ג\'יני, 0= שוויון מלא)': 'employ_inequality_gini_salaried',

        ### EDUCATION - KINDERGARTEN (edu_kinder_) ###
        'חינוך והשכלה - תשע"ח 2017/18 - ילדים בגנים של משרד החינוך - סה"כ': 'edu_kinder_total',
        'חינוך והשכלה - תשע"ח 2017/19 - ילדים בגנים של משרד החינוך - גיל 3': 'edu_kinder_age_3',
        'חינוך והשכלה - תשע"ח 2017/20 - ילדים בגנים של משרד החינוך - גיל 4': 'edu_kinder_age_4',
        'חינוך והשכלה - תשע"ח 2017/21 - ילדים בגנים של משרד החינוך - גיל 5': 'edu_kinder_age_5',
        'חינוך והשכלה - תשע"ח 2017/22 - ילדים בגנים של משרד החינוך - גיל 6': 'edu_kinder_age_6',

        ### EDUCATION - SCHOOLS (edu_schools_) ###
        'חינוך והשכלה - תשע"ח 2017/18 - בתי ספר - סה"כ': 'edu_schools_total',
        'חינוך והשכלה - תשע"ח 2017/19 - בתי ספר - יסודיים': 'edu_schools_elementary',
        'חינוך והשכלה - תשע"ח 2017/20 - בתי ספר - על יסודיים': 'edu_schools_highschool',
        'חינוך והשכלה - תשע"ח 2017/21 - בתי ספר - חטיבות ביניים': 'edu_schools_middle',
        'חינוך והשכלה - תשע"ח 2017/22 - בתי ספר - תיכוניים': 'edu_schools_secondary',

        ### EDUCATION - CLASSES (edu_classes_) ###
        'חינוך והשכלה - תשע"ח 2017/18 - כיתות - סה"כ': 'edu_classes_total',
        'חינוך והשכלה - תשע"ח 2017/19 - כיתות - בבתי ספר יסודיים': 'edu_classes_elementary',
        'חינוך והשכלה - תשע"ח 2017/20 - כיתות - בבתי ספר על-יסודיים': 'edu_classes_highschool',
        'חינוך והשכלה - תשע"ח 2017/21 - כיתות - בחטיבות ביניים': 'edu_classes_middle',
        'חינוך והשכלה - תשע"ח 2017/22 - כיתות - בבתי ספר תיכוניים': 'edu_classes_secondary',

        ### EDUCATION - STUDENTS (edu_students_) ###
        'חינוך והשכלה - תשע"ח 2017/18 - תלמידים - סה"כ': 'edu_students_total',
        'חינוך והשכלה - תשע"ח 2017/19 - תלמידים - בבתי ספר יסודיים': 'edu_students_elementary',
        'חינוך והשכלה - תשע"ח 2017/20 - תלמידים - בבתי ספר על-יסודיים': 'edu_students_highschool',
        'חינוך והשכלה - תשע"ח 2017/21 - תלמידים - בחטיבות ביניים': 'edu_students_middle',
        'חינוך והשכלה - תשע"ח 2017/22 - תלמידים - בבתי ספר תיכוניים': 'edu_students_secondary',

        ### EDUCATION - STUDENTS PER CLASS (edu_class_size_) ###
        'חינוך והשכלה - תשע"ח 2017/18 - ממוצע תלמידים לכיתה - סה"כ': 'edu_class_size_total_avg',
        'חינוך והשכלה - תשע"ח 2017/19 - ממוצע תלמידים לכיתה - בבתי ספר יסודיים': 'edu_class_size_elementary_avg',
        'חינוך והשכלה - תשע"ח 2017/20 - ממוצע תלמידים לכיתה - בבתי ספר על-יסודיים': 'edu_class_size_highschool_avg',
        'חינוך והשכלה - תשע"ח 2017/21 - ממוצע תלמידים לכיתה - בחטיבות ביניים': 'edu_class_size_middle_avg',
        'חינוך והשכלה - תשע"ח 2017/22 - ממוצע תלמידים לכיתה - בבתי ספר תיכוניים': 'edu_class_size_secondary_avg',

        ### EDUCATION - ACHIEVEMENTS (edu_achieve_) ###
        'חינוך והשכלה - תשע"ח 2017/18 - אחוז זכאים לתעודת בגרות מבין תלמידי כיתות יב': 'edu_achieve_bagrut_pct',
        'חינוך והשכלה - תשע"ח 2017/18 - אחוז זכאים לתעודת בגרות שעמדו בדרישות הסף של האוניברסיטאות מבין תלמידי כיתות יב': 'edu_achieve_bagrut_univ_pct',

        ### EDUCATION - HIGHER EDUCATION (edu_higher_) ###
        'חינוך והשכלה - 2019 - השכלה גבוהה - אחוז בעלי תארים מישראל מתוך אוכלוסיית בני 55-35': 'edu_higher_degree_holders_35_55_pct',
        'חינוך והשכלה - 2019 - השכלה גבוהה - אחוז הנכנסים להשכלה גבוהה בתוך 8 שנים בקרב תלמידי יב': 'edu_higher_enrollment_8years_pct',
        'חינוך והשכלה - 2019 - השכלה גבוהה - אחוז סטודנטים מתוך אוכלוסיית בני 25-20': 'edu_higher_students_20_25_pct',
        'חינוך והשכלה - 2019 - השכלה גבוהה - אחוז סטודנטים מתוך כלל האוכלוסייה': 'edu_higher_students_total_pct',

        ### EDUCATION - TEACHING STAFF (edu_staff_) ###
        'חינוך והשכלה - עובדי הוראה - מספר עובדי הוראה': 'edu_staff_total',
        'חינוך והשכלה - עובדי הוראה - ממוצע שעות עבודה לשבוע': 'edu_staff_weekly_hours_avg',
        'חינוך והשכלה - עובדי הוראה - אחוז עובדי הוראה חדשים': 'edu_staff_new_teachers_pct',
        'חינוך והשכלה - עובדי הוראה - אחוז שעות היעדרות מסך שעות עבודה שנתיות': 'edu_staff_absence_hours_pct',
        'חינוך והשכלה - עובדי הוראה - ממוצע שנות ותק מוכרות בהוראה': 'edu_staff_seniority_years',
        'חינוך והשכלה - עובדי הוראה - אחוז בעלי דרגת שכר תואר שני ומעלה': 'edu_staff_masters_degree_pct',
        'חינוך והשכלה - עובדי הוראה - אחוז עובדי הוראה משתלמים מכלל עובדי הוראה': 'edu_staff_training_pct',
        'חינוך והשכלה - עובדי הוראה - ממוצע תלמידים למורה': 'edu_staff_students_per_teacher',
        'חינוך והשכלה - עובדי הוראה - ממוצע שעות עבודה שבועיות לתלמיד': 'edu_staff_weekly_hours_per_student',

        ### WATER USAGE (water_) ###
        'מים - תקבולי מים (אלפי מ"ק)': 'water_supply_cubic_m',
        'מים - פחת מים סה"כ (אלפי מ"ק)': 'water_loss_total_cubic_m',
        'מים - אחוז פחת המים מסך כל תקבולי המים': 'water_loss_pct',
        'מים - סה"כ צריכת מים (אלפי מ"ק)': 'water_consumption_total_cubic_m',
        'מים - צריכת מים ביתית וכל צריכה אחרת (אלפי מ"ק)': 'water_consumption_residential_other_cubic_m',
        'מים - צריכת מים חקלאית (אלפי מ"ק)': 'water_consumption_agriculture_cubic_m',
        'מים - אחוז צריכת מים ביתית וכל צריכה אחרת מסה"כ צריכת המים': 'water_consumption_residential_other_pct',
        'מים - אחוז צריכת מים חקלאית מסה"כ צריכת המים': 'water_consumption_agriculture_pct',

        ### WATER USAGE BY TYPE (water_type_) ###
        'מים - צריכת מים ביתית (אלפי מ"ק) - למגורים': 'water_type_residential_cubic_m',
        'מים - צריכת מים ביתית (אלפי מ"ק) - למוסדות הרשות המקומית': 'water_type_municipal_cubic_m',
        'מים - צריכת מים ביתית (אלפי מ"ק) - לגינון ציבורי': 'water_type_gardening_cubic_m',
        'מים - צריכת מים ביתית (אלפי מ"ק) - למסחר ומלאכה': 'water_type_commercial_cubic_m',
        'מים - צריכת מים ביתית (אלפי מ"ק) - לבנייה': 'water_type_construction_cubic_m',

        ### WATER USAGE PERCENTAGES (water_type_pct_) ###
        'מים - אחוז צריכת מים מסה"כ צריכת מים ביתית - למגורים': 'water_type_pct_residential',
        'מים - אחוז צריכת מים מסה"כ צריכת מים ביתית - למוסדות הרשות המקומית': 'water_type_pct_municipal',
        'מים - אחוז צריכת מים מסה"כ צריכת מים ביתית - לגינון ציבורי': 'water_type_pct_gardening',
        'מים - אחוז צריכת מים מסה"כ צריכת מים ביתית - למסחר ומלאכה': 'water_type_pct_commercial',
        'מים - אחוז צריכת מים מסה"כ צריכת מים ביתית - לבנייה': 'water_type_pct_construction',

        ### WATER CONSUMPTION PER CAPITA (water_capita_) ###
        'מים - צריכת מים ביתית (מ"ק לנפש) - סה"כ': 'water_capita_total',
        'מים - צריכת מים ביתית (מ"ק לנפש) - למגורים': 'water_capita_residential',
        'מים - צריכת מים ביתית (מ"ק לנפש) - למוסדות הרשות המקומית': 'water_capita_municipal',
        'מים - צריכת מים ביתית (מ"ק לנפש) - לגינון ציבורי': 'water_capita_gardening',
        'מים - צריכת מים ביתית (מ"ק לנפש) - למסחר ומלאכה': 'water_capita_commercial',
        'מים - צריכת מים ביתית (מ"ק לנפש) - לבנייה': 'water_capita_construction',

        ### CONSTRUCTION AND HOUSING (construct_) ###
        'בנייה ודיור - התחלת בנייה - שטח (אלפי מ"ר)': 'construct_start_area_total_sqm',
        'בנייה ודיור - שטח למגורים (אלפי מ"ר)': 'construct_start_residential_sqm', 
        'בנייה ודיור - מספר דירות': 'construct_start_units_count',
        'בנייה ודיור - גמר בנייה - שטח (אלפי מ"ר)': 'construct_complete_area_total_sqm',
        'בנייה ודיור - שטח למגורים (אלפי מ"ר).1': 'construct_complete_residential_sqm',
        'בנייה ודיור - מספר דירות.1': 'construct_complete_units_count',
        
        ### HOUSING STOCK (housing_) ###
        'בנייה ודיור - יוני 2019 - מספר דירות למגורים לפי מרשם מבנים ודירות': 'housing_units_total',
            
        ### INFRASTRUCTURE - ROADS AND UTILITIES (infra_) ###
        'בנייה ודיור - התחלה של סלילת כבישים חדשים, הרחבה ושיקום של כבישים - אורך (ק"מ)': 'infra_roads_start_km',
        'בנייה ודיור - שטח (אלפי מ"ר)': 'infra_roads_start_area_sqm',
        'בנייה ודיור - התחלה של הנחת צינורות (ק"מ) - מים': 'infra_pipes_start_water_km',
        'בנייה ודיור - ביוב': 'infra_pipes_start_sewage_km',
        'בנייה ודיור - תיעול': 'infra_pipes_start_drainage_km',
        'בנייה ודיור - גמר של סלילת כבישים חדשים, הרחבה ושיקום של כבישים - אורך (ק"מ)': 'infra_roads_complete_km',
        'בנייה ודיור - שטח (אלפי מ"ר).1': 'infra_roads_complete_area_sqm',

        ### INFRASTRUCTURE - COMPLETED PIPES (infra_pipes_complete_) ###
        'בנייה ודיור - גמר של הנחת צינורות (ק"מ) - מים': 'infra_pipes_complete_water_km',
        'בנייה ודיור - ביוב.1': 'infra_pipes_complete_sewage_km',
        'בנייה ודיור - תיעול.1': 'infra_pipes_complete_drainage_km',

        ### WASTE MANAGEMENT (waste_) ###
        'פסולת מוצקה ביתית, מסחרית וגזם (ק"ג ליום לנפש) - פסולת מוצקה ביתית, מסחרית וגזם (ק"ג ליום לנפש)': 'waste_per_capita_daily_kg',
        'פסולת מוצקה ביתית, מסחרית וגזם (ק"ג ליום לנפש) - אחוז מחזור מתוך פסולת מוצקה ביתית, מסחרית וגזם': 'waste_recycling_pct',

        ### TRANSPORTATION (transport_) ###
        'תחבורה - כלי רכב מנועיים סה"כ': 'transport_vehicles_total',
        'תחבורה - כלי רכב פרטיים סה"כ': 'transport_private_vehicles_total',
        'תחבורה - גיל ממוצע של רכב פרטי (שנים)': 'transport_private_vehicles_age_avg',
        'תחבורה - תאונות דרכים עם נפגעים סה"כ': 'transport_accidents_total',
        'תחבורה - תאונות דרכים עם נפגעים - קטלנית': 'transport_accidents_fatal',
        'תחבורה - קשה': 'transport_accidents_severe',
        'תחבורה - קלה': 'transport_accidents_light',
        'תחבורה - תאונות דרכים עם נפגעים שיעור ל-1,000 תושבים': 'transport_accidents_rate_per_1000_residents',
        'תחבורה - תאונות דרכים עם נפגעים שיעור ל-1,000 כלי רכב': 'transport_accidents_rate_per_1000_vehicles',
        'תחבורה - הרוגים': 'transport_casualties_fatal',

        ### CRIME AND LAW (crime_) ###
        'פשיעה ומשפט - 2017 - מבוגרים תושבי ישראל המורשעים בדין, לפי רשות מקומית שבה גרו (רשויות עם 50,000 תושבים ויותר) - מספרים מוחלטים': 'crime_convictions_total',
        'פשיעה ומשפט - 2017 - שיעור ל-1,000 בני 19 ומעלה': 'crime_convictions_rate_per_1000',

        ### CRIME BY TYPE (crime_type_) ###
        'פשיעה ומשפט - 2017 - מורשעים בדין לפי סוג עבירה באחוזים - עבירות כלפי הסדר הציבורי': 'crime_type_public_order_pct',
        'פשיעה ומשפט - 2017 - עבירות כלפי חיי אדם וגופו': 'crime_type_violence_pct',
        'פשיעה ומשפט - 2017 - עבירות מוסר': 'crime_type_morality_pct',
            
        ### CRIME BY TYPE (crime_type_) ###
        'פשיעה ומשפט - 2017 - עבירות רכוש': 'crime_type_property_pct',
        'פשיעה ומשפט - 2017 - עבירות מרמה ועבירות כלכליות': 'crime_type_fraud_economic_pct',
        'פשיעה ומשפט - 2017 - עבירות רישוי': 'crime_type_licensing_pct',
        'פשיעה ומשפט - 2017 - אחר': 'crime_type_other_pct',

        ### SOCIO-ECONOMIC INDEX (socio_) ###
        'מדד חברתי-כלכלי - 2015 - אשכול (מ-1 עד 10, 1 הנמוך ביותר)': 'socio_cluster',
        'מדד חברתי-כלכלי - 2015 - ערך מדד': 'socio_index_value',
        'מדד חברתי-כלכלי - 2015 - דירוג (מ-1 עד 255, 1 הנמוך ביותר)': 'socio_rank',

        ### COMPACTNESS INDEX (compact_) ###
        'מדד קומפקטיות (עבור עיריות ומועצות מקומיות) - 2018 - אשכול (מ-1 עד 10, 1 הקומפקטי ביותר)': 'compact_cluster',
        'מדד קומפקטיות (עבור עיריות ומועצות מקומיות) - 2018 - דירוג (מ-1 עד 197, 1 הקומפקטי ביותר)': 'compact_rank',
        'מדד קומפקטיות (עבור עיריות ומועצות מקומיות) - 2018 - ציון תקן': 'compact_std_score',

        ### PERIPHERALITY INDEX (periph_) ###
        'מדד פריפריאליות - 2015 - אשכול (מ-1 עד 10, 1 הפריפריאלי ביותר)': 'periph_cluster',
        'מדד פריפריאליות - 2015 - ערך מדד': 'periph_index_value',
        'מדד פריפריאליות - 2015 - דירוג (מ-1 עד 255, 1 הפריפריאלי ביותר)': 'periph_rank',
        'מדד פריפריאליות - 2015 - דירוג לפי מרכיבי המדד: מדד נגישות פוטנציאלית (מ-1 עד 255, 1 הנגישות הנמוכה ביותר)': 'periph_accessibility_rank',
        'מדד פריפריאליות - 2015 - דירוג לפי מרכיבי המדד: קרבה לגבול מחוז תל אביב (מ-1 עד 255, 1 המרוחק ביותר)': 'periph_distance_tlv_rank',

        ### LAND USAGE TOTAL (land_total_) ###
        'שימושי קרקע - 2013 - סך הכל שטח שיפוט בקמ"ר': 'land_total_area_km2',
        'שימושי קרקע - 2013 - מגורים - שטח בקמ"ר': 'land_total_residential_km2',
        'שימושי קרקע - 2013 - מגורים - אחוז מסך כל שטח שיפוט': 'land_total_residential_pct',

        ### LAND USAGE BY TYPE (land_use_) ###
        'שימושי קרקע - 2013 - חינוך והשכלה - שטח בקמ"ר': 'land_use_education_km2',
        'שימושי קרקע - 2013 - חינוך והשכלה - אחוז מסך כל שטח שיפוט': 'land_use_education_pct',
        'שימושי קרקע - 2013 - בריאות ורווחה - שטח בקמ"ר': 'land_use_health_welfare_km2',
        'שימושי קרקע - 2013 - בריאות ורווחה - אחוז מסך כל שטח שיפוט': 'land_use_health_welfare_pct',
        'שימושי קרקע - 2013 - שירותים ציבוריים - שטח בקמ"ר': 'land_use_public_services_km2',
        'שימושי קרקע - 2013 - שירותים ציבוריים - אחוז מסך כל שטח שיפוט': 'land_use_public_services_pct',
        'שימושי קרקע - 2013 - תרבות, פנאי, נופש וספורט - שטח בקמ"ר': 'land_use_culture_recreation_km2',
        'שימושי קרקע - 2013 - תרבות, פנאי, נופש וספורט - אחוז מסך כל שטח שיפוט': 'land_use_culture_recreation_pct',
        'שימושי קרקע - 2013 - מסחר ומשרדים - שטח בקמ"ר': 'land_use_commerce_offices_km2',
        'שימושי קרקע - 2013 - מסחר ומשרדים - אחוז מסך כל שטח שיפוט': 'land_use_commerce_offices_pct',
        'שימושי קרקע - 2013 - תעשייה - שטח בקמ"ר': 'land_use_industry_km2',
        'שימושי קרקע - 2013 - תעשייה - אחוז מסך כל שטח שיפוט': 'land_use_industry_pct',
        'שימושי קרקע - 2013 - תשתית ותחבורה - שטח בקמ"ר': 'land_use_infrastructure_transport_km2',
        'שימושי קרקע - 2013 - תשתית ותחבורה - אחוז מסך כל שטח שיפוט': 'land_use_infrastructure_transport_pct',
        'שימושי קרקע - 2013 - מבנים חקלאיים - שטח בקמ"ר': 'land_use_agriculture_buildings_km2',
        'שימושי קרקע - 2013 - מבנים חקלאיים - אחוז מסך כל שטח שיפוט': 'land_use_agriculture_buildings_pct',
        'שימושי קרקע - 2013 - גינון לנוי פארק ציבורי - שטח בקמ"ר': 'land_use_public_parks_km2',
        'שימושי קרקע - 2013 - גינון לנוי פארק ציבורי - אחוז מסך כל שטח שיפוט': 'land_use_public_parks_pct',
        'שימושי קרקע - 2013 - יער וחורש - שטח בקמ"ר': 'land_use_forest_km2',
        'שימושי קרקע - 2013 - יער וחורש - אחוז מסך כל שטח שיפוט': 'land_use_forest_pct',

        ### LAND USAGE BY TYPE (land_use_) - ###
        'שימושי קרקע - 2013 - מטעים - שטח בקמ"ר': 'land_use_orchards_km2',
        'שימושי קרקע - 2013 - מטעים - אחוז מסך כל שטח שיפוט': 'land_use_orchards_pct',
        'שימושי קרקע - 2013 - גידול שדה - שטח בקמ"ר': 'land_use_field_crops_km2',
        'שימושי קרקע - 2013 - גידול שדה - אחוז מסך כל שטח שיפוט': 'land_use_field_crops_pct',
        'שימושי קרקע - 2013 - שטח פתוח אחר - שטח בקמ"ר': 'land_use_other_open_km2',
        'שימושי קרקע - 2014 - שטח פתוח אחר - אחוז מסך כל שטח שיפוט': 'land_use_other_open_pct',

        ### POPULATION DENSITY (density_) ### 
        'שימושי קרקע - 2013 - אוכלוסייה סוף 2013 סך הכל אלפים': 'density_population_thousands',
        'שימושי קרקע - 2013 - צפיפות אוכלוסייה לשטח בנוי למגורים - נפשות לקמ"ר': 'density_residential_per_km2',

        ### REGIONAL COUNCIL DISTANCES (region_dist_) ###
        'מרחקים (עבור מועצות אזוריות) - ממוצע המרחקים (ק"מ)': 'region_dist_avg_km',
        'מרחקים (עבור מועצות אזוריות) - ממוצע המרחקים משוקלל (ק"מ)': 'region_dist_weighted_avg_km',

        ### REGIONAL COUNCIL SETTLEMENTS (region_settle_) ###
        'יישובים (עבור מועצות אזוריות) - סה"כ יישובים במועצה': 'region_settle_total',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, יהודיים 5,000-9,999 תושבים': 'region_settle_urban_jewish_5k_10k',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, לא-יהודיים 9,999-5000 תושבים': 'region_settle_urban_nonjewish_5k_10k',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, יהודיים 2,000-4,999 תושבים': 'region_settle_urban_jewish_2k_5k',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, מושבים 2,000-4,999 תושבים': 'region_settle_urban_moshav_2k_5k',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, מושבים שיתופיים 2,000-4,999 תושבים': 'region_settle_urban_moshav_collective_2k_5k',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, קיבוצים 2,000-4,999 תושבים': 'region_settle_urban_kibbutz_2k_5k',
        'יישובים (עבור מועצות אזוריות) - יישובים עירוניים, לא-יהודיים 2,000-4,999 תושבים': 'region_settle_urban_nonjewish_2k_5k',
        'יישובים (עבור מועצות אזוריות) - מושבים (לרבות כפרים שיתופיים)': 'region_settle_moshav',
        'יישובים (עבור מועצות אזוריות) - מושבים שיתופיים': 'region_settle_moshav_collective',

        ### REGIONAL COUNCIL SETTLEMENTS (region_settle_) ###
        'יישובים (עבור מועצות אזוריות) - קיבוצים': 'region_settle_kibbutz',
        'יישובים (עבור מועצות אזוריות) - יישובים מוסדיים, יהודיים': 'region_settle_institutional_jewish',
        'יישובים (עבור מועצות אזוריות) - יישובים מוסדיים, לא-יהודיים': 'region_settle_institutional_nonjewish',
        'יישובים (עבור מועצות אזוריות) - יישובים קהילתיים': 'region_settle_community',
        'יישובים (עבור מועצות אזוריות) - יישובים כפריים אחרים, יהודיים': 'region_settle_rural_other_jewish',
        'יישובים (עבור מועצות אזוריות) - יישובים כפריים אחרים, לא-יהודיים': 'region_settle_rural_other_nonjewish',

        ### ELECTIONS (elect_) ###
        'בחירות - בחירות מוניציפליות לראש ראשות - סבב א 30/10/18 - אחוזי הצבעה': 'elect_municipal_round1_turnout_pct',
        'בחירות - בחירות מוניציפליות לראש ראשות - סבב ב 13/11/18 - אחוזי הצבעה': 'elect_municipal_round2_turnout_pct',
        'בחירות - בחירות לכנסת ה-21 9/4/19 - אחוזי הצבעה': 'elect_knesset_21_turnout_pct',
        'בחירות - בחירות לכנסת ה-22 17/9/19 - אחוזי הצבעה': 'elect_knesset_22_turnout_pct'
        }
    return mapping

In [126]:
def standardize_columns(df):
   """Standardize DataFrame column names by cleaning and mapping Hebrew names to English.
   Also validates that no Hebrew characters remain in column names"""

   # Create a copy of the input DataFrame to avoid modifying the original
   df_clean = df.copy()
   
   # Rename columns using the cleaned mapping
   df_clean = df_clean.rename(columns = mapping)
   
   # Check for any remaining Hebrew characters in column names
   hebrew_pattern = r'[\u0590-\u05FF]'  # Range of Hebrew unicode characters
   hebrew_cols = [col for col in df_clean.columns if re.search(hebrew_pattern, col)]
   
   # Print results of the Hebrew character check
   if hebrew_cols:
       print("\nColumns still containing Hebrew characters:")
       for col in hebrew_cols:
           print(f"{col}")
   else:
       print("\nAll columns successfully converted to English")
       
   return df_clean


Columns still containing Hebrew characters:
תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות-הכנסות מארנונה- 'לא למגורים
(גבייה)
תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-הכנסות עצמיות - הכנסות מחינוך 
תקבולים בתקציב הרגיל - הכנסות לפי סיווג כלכלי-מלוות לאיזון 
עודף/גירעון-'עודף/גירעון
בתקציב הרגיל
השנה
עודף/גירעון-'עודף/גירעון בתקציב הרגיל
השנה
ללא קבלת מלוות לאיזון
שם  הרשות_y
כללי - מחוז 
כללי - 2015 - מרחק מגבול מחוז תל אביב (ק"מ)  
כללי - שנת קבלת מעמד מוניציפלי  
כללי - מרחב ימי, רשויות מקומיות הגובלות בחוף הים 
כללי - שטח 
(קמ"ר)
דמוגרפיה - סה"כ  אוכלוסייה בסוף השנה
דמוגרפיה - אחוז עולים
בהשתקעות ראשונה - בני 17-0
דמוגרפיה - אחוז עולים
בהשתקעות ראשונה - בני 44-18
דמוגרפיה - אחוז עולים
בהשתקעות ראשונה - בני 64-45
דמוגרפיה - אחוז עולים
בהשתקעות ראשונה - בני 65 ומעלה
דמוגרפיה - מאזן הגירה ביישוב
סה"כ
דמוגרפיה - נכנסים מיישובים אחרים
 - סה"כ
דמוגרפיה - נכנסים מיישובים אחרים
 - גברים
דמוגרפיה - נכנסים מיישובים אחרים
 - נשים
דמוגרפיה - נכנסים מיישובים אחרים
 - עד גיל 1

In [ ]:

df_english = standardize_columns(merged_df)


In [ ]:
# Save the  dataset
output_file_name = "df_2018_final.xlsx"
df_english.to_excel(output_file_name, index=False)

print(f"The DataFrame has been successfully exported to {output_file_name}.")